In [2]:
from data.graph_loader import GraphLoader
from data.gtfs_loader import GTFSLoader
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [3]:
graph_loader = GraphLoader()
graph_walk = graph_loader.create_graph_walk("data/graphs/ZMG_walk", "data/osm/ZMG_enclosure_2km.geojson")


Loading graph from data/graphs/ZMG_walk.pkl


In [4]:
gtfs_loader = GTFSLoader()
transit_df = gtfs_loader.load_transit_dataframe("data/gtfs")
stops_df = gtfs_loader.load_stops_dataframe("data/gtfs", transit_df)


In [5]:
def create_graph_transit(transit_df, stops_df):
    # create graph transit adjacency list in stops_df
    G = nx.MultiDiGraph()

    for idx, stop in stops_df.iterrows():
        stop_id = stop['stop_id']
        G.add_node(stop_id, pos=stop['geometry'], stop_name=stop['stop_name'], x=stop['stop_lon'], y=stop['stop_lat'], routes=stop['routes_by_stop'], shapes=stop['shapes_by_stop'])
        next_stops = stop['next_stop_id']
        for next_stop_id, edge_list in next_stops.items():
            for edge in edge_list:                
                G.add_edge(stop_id, next_stop_id, weight=edge['weight'], shape_id=edge['shape_id'], frequency=edge['frequency'])
    return G

graph_transit = create_graph_transit(transit_df, stops_df)

   

In [6]:
#get first edge in the walk graph
walk_gdf = ox.graph_to_gdfs(graph_walk, nodes=True, edges=False, fill_edge_geometry=True)
#create dataframe from walk_gdf
walk_df = pd.DataFrame(walk_gdf)


stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs='EPSG:4326')

In [ ]:
import heapq
from shapely import Point, LineString
#src and dst are Point
def euclidean_heuristic(src:Point, dst:Point) -> float:
    # Time to reache the dst
    # distance in meters using euclidean distance
    transit_average_speed_kph = 55.0  # average transit max speed in km/h
    transit_average_speed_mps = transit_average_speed_kph / 3.6  # convert to m/s    
    pts_m = gpd.GeoSeries([src, dst], crs="EPSG:4326").to_crs("EPSG:32613")
    distance_meters = pts_m.iloc[0].distance(pts_m.iloc[1])
    time_to_reach = distance_meters / transit_average_speed_mps  #time in s
    return round(time_to_reach)

def no_heuristic(src, dst):
    return 0
 

def check_no_transfers(graph:nx.MultiDiGraph, src, dst, transit_df, stops_df):

     #verify if src and dst share a shape, only one bus is taken
    src_shapes = set(stops_df[stops_df['stop_id'] == src]['shapes_by_stop'].iloc[0])
    dst_shapes = set(stops_df[stops_df['stop_id'] == dst]['shapes_by_stop'].iloc[0])

    shared_shapes = src_shapes & dst_shapes

    if shared_shapes:
        print("Shared shapes found:", shared_shapes)
        shape = shared_shapes.pop()
        shape_info = transit_df[transit_df['shape_id'] == shape].iloc[0]
        
        shape_stops = shape_info['stop_ids']
        src_index = shape_stops.index(src)
        dst_index = shape_stops.index(dst)
        if src_index < dst_index:
            path = shape_stops[src_index:dst_index + 1]
            stop_time_deltas = shape_info['stop_time_deltas']          
            total_cost = sum(stop_time_deltas[src_index:dst_index])
            
            #add to the path the shape info
            path = [(stop, shape) for stop in path]
            return path, total_cost
        
    return [], None

def check_one_transfer(graph:nx.MultiDiGraph, src, dst, transit_df, stops_df):
    
        #verif if src and dst shapes share a stop, only one transfer is needed
    src_shapes = set(stops_df[stops_df['stop_id'] == src]['shapes_by_stop'].iloc[0])
    dst_shapes = set(stops_df[stops_df['stop_id'] == dst]['shapes_by_stop'].iloc[0])

    for src_shape in src_shapes:
        src_shape_stops = transit_df[transit_df['shape_id'] == src_shape]['stop_ids'].iloc[0]
        for dst_shape in dst_shapes:
            dst_shape_stops = transit_df[transit_df['shape_id'] == dst_shape]['stop_ids'].iloc[0]
            #find common stops
            common_stops = set(src_shape_stops) & set(dst_shape_stops)
            if common_stops:
                transfer_stop = common_stops.pop()
                #build path from src to transfer_stop
                src_index = src_shape_stops.index(src)
                transfer_index_src = src_shape_stops.index(transfer_stop)
                path_src = src_shape_stops[src_index:transfer_index_src + 1]
                #build path from transfer_stop to dst
                transfer_index_dst = dst_shape_stops.index(transfer_stop)
                dst_index = dst_shape_stops.index(dst)
                path_dst = dst_shape_stops[transfer_index_dst:dst_index + 1]
                #combine paths
                full_path = path_src + path_dst[1:]  #avoid duplicating transfer_stop
                total_cost = 0
                for i in range(len(full_path) - 1):
                    edge_data = graph.get_edge_data(full_path[i], full_path[i + 1])
                    weight = edge_data.get('weight', 1)
                    total_cost += weight
                return full_path, total_cost    
    
    return [], None

def dijkstra(graph:nx.MultiDiGraph, src, dst, transit_df: pd.DataFrame, stops_df: pd.DataFrame, heuristic=euclidean_heuristic):
    #accumulated cost for each node from the start, keys are (node, shape_id)
    cost = dict()
    #stores the node and shape that it was coming from, keys are (node, shape_id)
    previous = dict()

    #"minimum heap" of the candidates (priorty_cost, node, shape_id)
    queue = []
    #to return the path from src to dst, list of tuples (node, shape_id)
    path = []
    
    path, total_cost = check_no_transfers(graph, src, dst, transit_df, stops_df)
    if path:
        print("No transfers needed.")
        return path, total_cost
    path, total_cost = check_one_transfer(graph, src, dst, transit_df, stops_df)
    if path:
        print("One transfer needed.")
        return path, total_cost      

    # fill the queue with all the possible starting edges from src
    for neighbor in graph.neighbors(src):
        edges = graph.get_edge_data(src, neighbor)
        if not edges:
            continue
        for k, attrs in edges.items():
            shape_id = attrs.get('shape_id')
            weight = attrs.get('weight', 1)
            frequency = attrs.get('frequency', 600)
            tentative_cost = weight + frequency #initial transfer penalty
            cost[(neighbor, shape_id)] = tentative_cost
            previous[(neighbor, shape_id)] = (src, None)
            heuristic_cost = heuristic(graph.nodes[neighbor]['pos'], graph.nodes[dst]['pos'])
            priority_cost = tentative_cost + heuristic_cost
            heapq.heappush(queue, (priority_cost, neighbor, shape_id))
    

    end_shape = None

    while queue:

        _, current, current_shape = heapq.heappop(queue)

        if current == dst:
            end_shape = current_shape
            break


        # explore all neighbors (MultiDiGraph may have parallel edges) and use min edge weight
        for neighbor in graph.neighbors(current):

            edges = graph.get_edge_data(current, neighbor)
            if not edges:
                continue
            
            for k, attrs in edges.items():
                next_shape = attrs.get('shape_id')
                weight = attrs.get('weight', 1)
                frequency = attrs.get('frequency', 600)
                
                if (current_shape is not None and next_shape is not None and next_shape != current_shape):
                    penalty = frequency
                else:
                    penalty = 0
                
                tentative_cost = cost[(current, current_shape)] + weight + penalty

                # if cost for neighbor with next_shape is not set or tentative_cost is lower, update
                if (neighbor, next_shape) not in cost or tentative_cost < cost[(neighbor, next_shape)]:
                    cost[(neighbor, next_shape)] = tentative_cost                
                    previous[(neighbor, next_shape)] = (current, current_shape)
                    heuristic_cost = heuristic(graph.nodes[neighbor]['pos'], graph.nodes[dst]['pos']) + 3*penalty
                    priority_cost = tentative_cost + heuristic_cost
                    heapq.heappush(queue, (priority_cost, neighbor, next_shape))
        
    if end_shape is None:
        print("No path found from", src, "to", dst)
        return [], None


    #recreate the path based on the previous going backwards but store it in forward 
    current = (dst, end_shape)

    while current in previous:
        path.append(current)
        current = previous.get(current)
    path.reverse()
    
    return path, cost[(dst, end_shape)]


In [12]:
#choose randomly a node from G
import random

path = []

while path == []:
    start = random.choice(list(graph_transit.nodes))
    destination = random.choice(list(graph_transit.nodes))

    try:
        path = nx.shortest_path(graph_transit, source=start, target=destination, weight='weight')
    except nx.NetworkXNoPath:
        path = []

print("Shortest path from", start, "to", destination, ":", path)


Shortest path from mxc_C98_STP_22 to mxc_C118_STP_48 : ['mxc_C98_STP_22', 'mxc_C98_STP_23', 'mxc_C98_STP_24', 'mxc_C07V2_STP_01', 'mxc_C105_STP_09', 'mxc_C105_STP_11', 'mxc_C27_STP_20', 'mxc_C28_STP_11', 'mxc_C07V2_STP_07', 'mxc_C29V2_STP_18', 'mxc_C29V2_STP_19', 'mxc_C29V2_STP_20', 'mxc_C122_STP_89', 'mxc_C93_STP_76', 'mxc_C13V2_STP_08', 'mxc_C13V2_STP_09', 'mxc_C13V2_STP_10', 'mxc_C53_STP_179', 'mxc_C53_STP_180', 'mxc_C18_STP_24', 'mxc_C08_STP_75', 'mxc_C108V1_STP_25', 'mxc_C108V1_STP_26', 'mxc_C108V1_STP_27', 'mxc_C108V1_STP_28', 'mxc_C68_STP_61', 'mxc_C69_STP_33', 'mxc_C22_STP_38', 'mxc_C22_STP_39', 'mxc_C08_STP_32', 'mxd_mxc_AMG_T18_1_STP_65', 'mxd_mxc_AMG_T18_1_STP_66', 'mxc_C69_STP_39', 'mxd_mxc_AMG_T18_1_STP_68', 'mxd_mxc_AMG_T18_1_STP_69', 'mxd_mxc_AMG_T18_1_STP_70', 'mxd_mxc_AMG_T18_1_STP_71', 'mxd_mxc_AMG_T18_1_STP_72', 'mxc_C38V1_STP_56', 'mxc_C38V1_STP_57', 'mxc_C97V2_STP_67', 'mxd_mxc_AMG_T18_1_STP_76', 'mxc_C97V2_STP_69', 'mxc_C97V2_STP_70', 'mxd_mxc_AMG_T03_STP_43', 'mx

In [13]:
dijkstra_path, dijkstra_cost = dijkstra(graph_transit, start, destination, transit_df, stops_df)
print("Shortest path from", start, "to", destination, ":", dijkstra_path)

print("Cost of the path:", dijkstra_cost/60)

transit_gdf = gpd.GeoDataFrame(transit_df, geometry='shape_geometry', crs='EPSG:4326')

dijkstra_path_stops = []
dijkstra_path_shapes = []
prev_shape = None
for stop, shape in dijkstra_path:
    dijkstra_path_stops.append(stop)
    dijkstra_path_shapes.append(shape)
    if prev_shape is None:
        print("Walk to stop:", stop)
    if shape != prev_shape:
        print("Take Bus:", shape, "from stop:", stop, " direction to:", transit_df[transit_df['shape_id'] == shape]['trip_headsign'].iloc[0])
        prev_shape = shape


#show all the shapes in the path
#assign colors based on shape_id or randomly
shapes_in_path = sorted(set(dijkstra_path_shapes))
subset = transit_gdf[transit_gdf['shape_id'].isin(shapes_in_path)]
m = subset.explore(
    column='route_short_name',
    cmap='tab20',
    legend=True,
    tooltip=['route_long_name', 'route_short_name', 'trip_headsign']
)
print(set(shapes_in_path))

path_gdf = stops_gdf[stops_gdf['stop_id'].isin(dijkstra_path_stops)]
m = path_gdf.explore(color='red', m=m)
openMap(m)

Shortest path from mxc_C98_STP_22 to mxc_C118_STP_48 : [('mxc_C98_STP_23', 'C98_r1'), ('mxc_C98_STP_24', 'C98_r1'), ('mxc_C07V2_STP_01', 'C98_r1'), ('mxc_C98_STP_26', 'C98_r1'), ('mxc_C105_STP_18', 'C98_r1'), ('mxc_C07V2_STP_11', 'C98_r1'), ('mxc_C07V2_STP_12', 'C98_r1'), ('mxc_C07V2_STP_13', 'C98_r1'), ('mxc_C07V2_STP_14', 'C98_r1'), ('mxc_C07V1_STP_08', 'C98_r1'), ('mxc_C07V1_STP_09', 'C98_r1'), ('mxc_C29V1_STP_05', 'C29-V1_r1'), ('mxc_C29V1_STP_06', 'C29-V1_r1'), ('mxc_C29V1_STP_07', 'C29-V1_r1'), ('mxc_C29V1_STP_08', 'C29-V1_r1'), ('mxc_C108V2_STP_136', 'C29-V1_r1'), ('mxc_C108V2_STP_137', 'C29-V1_r1'), ('mxc_C29V1_STP_11', 'C29-V1_r1'), ('mxc_C108V2_STP_139', 'C29-V1_r1'), ('mxc_C108V2_STP_140', 'C29-V1_r1'), ('mxc_C29V1_STP_14', 'C29-V1_r1'), ('mxc_C29V1_STP_15', 'C29-V1_r1'), ('mxc_C108V2_STP_143', 'C29-V1_r1'), ('mxc_C108V2_STP_144', 'C29-V1_r1'), ('mxc_C108V2_STP_145', 'C29-V1_r1'), ('mxc_C108V2_STP_01', 'C29-V1_r1'), ('mxc_C108V2_STP_02', 'C29-V1_r1'), ('mxc_C108V2_STP_03', '